##  Реализация web-crawler для сбора коллекции документов с указанного ресурса. 

Выбор формата хранения коллекции - MongoDB? Forlder with files? Relation DataBase

### Характеристики коллекции:
- Объем собранной коллекции не менее 100 тысяч уникальных документов 
- Размер текста для каждого документа должен быть не менее 2000 символов
- Механизм очистки документов от не релевантной информации, например:
    - html теги
    - ссылки на сторонние или внутренние ресурсы сайта
    - ссылки на изображения и видео
    - знаки препинания
- Выделение признаков документа
    - автор
    - тематические теги
    - рейтинг
    - репосты
    - дата публикации 
    - другая метаинформация, предоставляемая ресурсом.

### Результатом сдачи является:
- код web-crawler
- инструмент очистки текстов
- исходная коллекция документов
- коллекция документов после очистки



In [1]:
PIKABU = "https://pikabu.ru/"
PIKABU_HOT = "https://pikabu.ru/"
PIKABU_BEST = "https://pikabu.ru/best"
PIKABU_FRESH = "https://pikabu.ru/new"
PIKABU_GROUP = "https://pikabu.ru/communities"

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [3]:
import html5lib

In [90]:
import requests
from lxml import html

### Парсинг через BeautifulSoup

In [22]:
path_webdriver = 'C:/Users/Public/Programms/chromedriver_win32/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')  # для открытия headless-браузера

browser = webdriver.Chrome(executable_path=path_webdriver, chrome_options=options)
browser.get(PIKABU);

c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [23]:
main_page = browser.page_source

In [24]:
print(type(main_page))

<class 'str'>


In [69]:
soup = BeautifulSoup(main_page, 'lxml')
table = soup.findChildren('article')
# table = soup.findChildren('div', {'class': 'story__content-inner'})

print(len(table))
# print(table[0].prettify())

# print(table[2].find('div', {'class': 'story__content-inner'}))

15


In [161]:
root = table[0]

# перебор всех тегов

recursive_childs = [e.name for e in root.recursiveChildGenerator() if e.name is not None]  
print(recursive_childs)


# При помощи атрибута children можно вывести все дочерние теги.

root_childs = [e.name for e in root.children if e.name is not None]
print(root_childs)


# список всех потомков (дочерних элементов всех уровней) рассматриваемого тега.

root_childs = [e.name for e in root.descendants if e.name is not None]
# print(root_childs)

['div', 'div', 'div', 'div', 'svg', 'use', 'div', 'div', 'svg', 'use', 'div', 'div', 'div', 'span', 'div', 'header', 'h2', 'a', 'div', 'div', 'div', 'p', 'p', 'div', 'a', 'a', 'a', 'a', 'a', 'div', 'div', 'a', 'span', 'svg', 'use', 'span', 'div', 'span', 'svg', 'use', 'span', 'div', 'svg', 'use', 'div', 'svg', 'use', 'div', 'div', 'div', 'div', 'a', 'div', 'time', 'a', 'div', 'span', 'img']
['div', 'div']


In [89]:
print(table[2].find('div', {'class': 'story__tags tags'}).text)


[моё] Коронавирус Общепит Кризис Список Деньги



In [160]:
# print(root.prettify())

### Парсинг через requests

как получить user-agent описано здесь https://habr.com/ru/post/280238/

In [75]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 YaBrowser/20.3.1.197 Yowser/2.5 Safari/537.36'
}
r = requests.get('https://pikabu.ru//', headers = headers)
print(r)

<Response [200]>


In [77]:
soup = BeautifulSoup(r.text, 'lxml')
articles = soup.find('div', {'class': 'story__content-inner'})
print(articles)
# lxml 
tree = html.fromstring(r.text)
#film_list_lxml = tree.xpath('//div[@class = "profileFilmsList"]')[0]

<div class="story__content-inner">
<div class="story-block story-block_type_text"><p>Заходит как-то Путин в бар и говорит:</p><p>- Всем пива за счет заведения!</p></div>
</div>


In [ ]:
print(soup.find("ul", id="mylist"))



In [99]:
# удаление тегов
test = BeautifulSoup(r.text, 'lxml')
div = test.select_one("head")
div.decompose()
div = test.select_one("body")
div.decompose()
print(test.prettify())

<!DOCTYPE html>
<html class="no-svg no-js" dir="ltr" lang="ru">
</html>



In [159]:
# удаление всех тегов подобного типа

test = BeautifulSoup(r.text, 'lxml')
div = test.select("meta")
for d in div:
    d.decompose()
div = test.select("link")
for d in div:
    d.decompose()
# print(test.prettify())

In [158]:
def get_html(url):
    r = requests.get(url, headers=headers)    # Получаем метод Response
    return r.text            # Вернем данные объекта text

# print(get_html(PIKABU))

def get_head(html):
    soup = BeautifulSoup(html, 'lxml') #  Создаем сам объект , передаем в него наш код страницы (html) 
    return soup
    
soup = get_head(get_html(PIKABU))

# print(soup.prettify())
## Теперь с помощью метода fine() найдем блок со статьями, <div id="section-content"> . 
## Уже в этом блоке найдем все теги заголовка <h1> методом fine_all() в которых собственно и содержится название статьи.

head = soup.find_all('div', {'class':'auth__field'})

In [157]:
# print(head)

In [136]:
def save_html(html_page, filename):
    html_file = open(filename, "w", encoding='UTF-8')
    html_file.write(html_page)
    html_file.close()
    
save_html(soup.prettify(),"mainpage.html")    

In [138]:
class Article():
    
    def __init__(self, _title, _author, _date, _text, _rating, _tags, _amount_comments, _have_image):
        self.title = _title
        self.author = _author
        self.date = _date
        self.text = _text
        self.rating = _rating
        self.tags = _tags
        self.amount_comments = _amount_comments
        self.have_image = _have_image
        
    def showInfo():
        pass

In [139]:
art = Article("title","Sabina","30.03","TestJsonObjectSerialize",100,['tag1', 'tag2'], 10, False)

In [140]:
import json

In [156]:
s = json.dumps(art.__dict__) 
# print(json.dumps(art.__dict__, sort_keys=True, indent=4))

In [145]:
# here we create new data_file.json file with write mode using file i/o operation 
with open('json_file.json', "w") as file_write:
    # write json data into file
    json.dump(art.__dict__, file_write)

In [146]:
with open('json_file.json') as file_object:
        # store file data in object
    data = json.load(file_object)
    print(data)

{'title': 'title', 'author': 'Sabina', 'date': '30.03', 'text': 'TestJsonObjectSerialize', 'rating': 100, 'tags': ['tag1', 'tag2'], 'amount_comments': 10, 'have_image': False}


In [155]:
with open('json_file.json') as file_object:
        # store file data in object
    data = json.load(file_object)
    print(data['title'])

title
